In [ ]:
#整个过程是参考的tensorflow BERT官网实现https://github.com/google-research/bert，
#以及一个外国工程师基于官网实现做的horovod分布式处理BERT finetuning的实现https://github.com/lambdal/bert

#整个过程包括几个步骤：
#1. 准备数据
#2. 准备预训练好的BERT模型
#3. 准备clone github的代码
#4. 在本地测试基于BERT模型的finetuning的文本分类任务。
#5. 将代码迁移到Sagemaker中，并利用horovod进行多机多卡的分布式训练

#1. 利用官网提供的脚本来下载数据
#步骤是：下载脚本；利用这个脚本下载数据；解压数据

In [ ]:
!wget https://gist.github.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e/archive/17b8dd0d724281ed7c3b2aeeda662b92809aadd5.zip

In [ ]:
!unzip 17b8dd0d724281ed7c3b2aeeda662b92809aadd5.zip

In [ ]:
!python 60c2bdb54d156a41194446737ce03e2e-17b8dd0d724281ed7c3b2aeeda662b92809aadd5/download_glue_data.py --data_dir glue_data --tasks all

In [ ]:
#2. 准备预训练好的BERT模型
#这里使用的是全部小写化的large模型：下载预训练的BERT模型并解压

In [ ]:
!wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

In [ ]:
!unzip uncased_L-12_H-768_A-12.zip

In [ ]:
#3. 准备clone github的代码
#在sagemaker jupyter lab中对应的项目目录中创建source_code目录，
#然后选择git按钮输入https://github.com/lambdal/bert.git

In [ ]:
%%sh
mkidr source_code
cd source_code
git clone https://github.com/lambdal/bert.git

In [ ]:
#4. 在本地测试基于BERT模型的finetuning的文本分类任务。
#注意下面的一些参数： data_dir是需要使用的数据集，voca_file是需要用到的词典（来自上一步解压后的一个文件）
#                  bert_config_file是bert的配置文件，init_checkpoint是预训练好的BERT模型。
#                  output_dir是训练过程中的checkpoint保存的路径。

In [ ]:
!python ./source_code/bert/run_classifier.py \
  --task_name=MRPC \
  --do_train=true \
  --do_eval=true \
  --data_dir=./glue_data/MRPC \
  --vocab_file=./uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=./uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=./uncased_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=32 \
  --learning_rate=2e-5 \
  --num_train_epochs=1.0 \
  --output_dir=./mrpc_output/

In [ ]:
#5. 将代码迁移到Sagemaker中，并利用horovod进行多机多卡的分布式训练

In [ ]:
#5.1 把预训练的包解压完的所有文件拷贝到source_code/bert目录下。
#这样做的原因是在使用Sagemaker BYOS来训练的时候，训练脚本依赖的所有source code，配置文件，资源文件都需要和训练脚本在同一级的目录。

In [ ]:
!cp uncased_L-12_H-768_A-12/* source_code/bert/

In [ ]:
#5.2 把数据集上传到AWS的S3的桶（桶要提前建立）

In [ ]:
!aws s3 sync glue_data/MRPC s3://sagemaker-us-west-2-169088282855/bert-SM-TF-test

In [ ]:
#5.3 把训练脚本以及依赖的配置文件，资源文件（这里指的是预训练的模型文件）上传到S3的桶
#其实不上传也没有问题，但是每次在开始训练前sagemaker都要重新对这些文件打包上传，影响训练启动速度。
#因此最好的方式就是提前打包上传到S3.
#把原来bert目录下的requirements.txt文件换个名字，否则Sagemaker会安装这个文件中的软件包（这里我们不需要安装任何软件包了）

In [ ]:
!mv source_code/bert/requirements.txt source_code/bert/requirements.txt.copy

In [ ]:
%%script bash
cd source_code/bert
tar cvfz sourcedir.tar.gz *
aws s3 cp sourcedir.tar.gz s3://sagemaker-us-west-2-169088282855/bert-hvd-tf/

In [ ]:
#5.4 调用Sagemaker的API开始finetuning

In [ ]:
#这里是使用horovod多机多卡训练，使用的是ml.p3.8xlarge（4个GPU卡），所以需要设置每个实例的worker数量为4
#为了节省成本，这里使用spot实例来训练。
#参数output_dir需要设置为/opt/ml/model，只有在这个路径下，sagemaker训练完才会把这个路径下的所有文件打包并上传。
#参数data_dir需要设置为/opt/ml/input/data/training/，sagemaker file mode训练方式会把数据下载到这个目录，最后一级目录的名字（这里是training）需要和你的S3的channel的名字保持一致
#参数do_train和do_eval需要设置为1，不能写True或者true，因为SM中使用hyperparameter对自己的脚本输入参数的时候，SM会把所有的参数都字符串化。
#参数vocab_file，bert_config_file和init_checkpoint的路径前缀都是/opt/ml/code/，这个是因为sagemaker会把训练脚本以及所有依赖的源代码，配置文件，资源文件等都拷贝到这个目录。
#entry_point设置为run_classifier_hvd.py进行分布式训练，这个脚本中已经写成了horovod集成tensorflow的方式。
#source_dir参数指向指向打包代码以及依赖上传到S3的路径。
#train_volume_size设置大一些（不一定像我下面设置的1TB这么大），否则模型在训练过程中保存checkpoint或者在训练完打包这些checkpoint上传S3的时候可能硬盘空间不够。

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

train_instance_type = 'ml.p3.8xlarge'
hvd_processes_per_host = 4
distributions = {'mpi': {
                    'enabled': True,
                    'processes_per_host': hvd_processes_per_host,
                    'custom_mpi_options': '-verbose --NCCL_DEBUG=INFO -x OMPI_MCA_btl_vader_single_copy_mechanism=none'
                        }
                }

train_use_spot_instances = True
train_max_run=432000
train_max_wait = 432000 if train_use_spot_instances else None


hyperparameters = {'output_dir': '/opt/ml/model', 'data_dir': '/opt/ml/input/data/training/',
                   'do_train': 1, 'do_eval': 1,
                   'vocab_file': '/opt/ml/code/vocab.txt', 
                   'bert_config_file': '/opt/ml/code/bert_config.json', 
                   'init_checkpoint': '/opt/ml/code/bert_model.ckpt',
                   'num_train_epochs': 100.0, 'train_batch_size': 32, 'max_seq_length': 128, 'learning_rate': 2e-5, 
                   'task_name':'MRPC'}

estimator = TensorFlow(entry_point='run_classifier_hvd.py',
                       source_dir='s3://sagemaker-us-west-2-169088282855/bert-hvd-tf/sourcedir.tar.gz',
                       train_instance_type=train_instance_type,
                       train_instance_count=2,
                       train_volume_size=1000,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-bert-sm-test',
                       framework_version='1.14',
                       py_version='py3',
                       script_mode=True,
                       distributions=distributions,
                       train_use_spot_instances=train_use_spot_instances,
                       train_max_wait=train_max_wait,
                       train_max_run=train_max_run
                       )

In [ ]:
train_s3 = 's3://sagemaker-us-west-2-169088282855/bert-SM-TF-test'
train_channel = 'training'

inputs = {train_channel: train_s3}

estimator.fit(inputs)